In [1]:
import requests
import json
import pandas as pd
import numpy as np
import dotenv
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime,timedelta

# Adding Holiday Info

Reading holiday file

In [21]:
holidays = pd.read_csv('./data/NSW_holidays_2023.csv')
# Dropping unnecessary columns
holidays.drop('Unnamed: 0',axis=1,inplace=True)
# Changing format of date 
holidays['date'] = pd.to_datetime(holidays['date'],format='%b %d')
holidays['date'] = holidays['date'].dt.strftime('2023-%m-%d')


holidays

,Day,date,Holiday Name,Type,Comments
0,Sunday,2023-01-01,New Year's Day,National Holiday,NaN
1,Monday,2023-01-02,New Year's Day Holiday,National Holiday,NaN
2,Thursday,2023-01-26,Australia Day,National Holiday,NaN
3,Friday,2023-04-07,Good Friday,National Holiday,NaN
4,Saturday,2023-04-08,Easter Saturday,Regional Holiday,"Except TAS, WA"
5,Sunday,2023-04-09,Easter Sunday,Regional Holiday,"ACT, NSW, QLD, VIC, WA"
6,Monday,2023-04-10,Easter Monday,National Holiday,Monday after Easter Sunday
7,Tuesday,2023-04-25,Anzac Day,National Holiday,NaN
8,Sunday,2023-05-14,Mother's Day,Not A Public Holiday,2nd Sunday in May. Not a public holiday
9,Monday,2023-06-12,King's Birthday,Regional Holiday,"2nd Monday in June. except QLD, WA"


Creating bool columns to indicate whether day is a holiday

# COMBINED DATAFRAME WORK

File has been created with zones separated from column.

In [2]:
test = pd.read_parquet('./data/carpark_history_6_months_with_zones.parquet')
test.head()

,tsn,time,spots_x,zones,ParkID,occupancy,MessageDate,facility_id,facility_name,tfnsw_facility_id,parent_zone_id,spots_y,zone_id,zone_name,occupancy.loop,occupancy.monthlies,occupancy.open_gate,occupancy.total,occupancy.transients
0,207210,741448862,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T00:01:02,6,Gordon Henry St North Car Park,207210TPR001,0,213,1,Gordon Henry St North Car Park,None,None,None,31,None
1,207210,741450092,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T00:21:32,6,Gordon Henry St North Car Park,207210TPR001,0,213,1,Gordon Henry St North Car Park,None,None,None,30,None
2,207210,741468991,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T05:36:31,6,Gordon Henry St North Car Park,207210TPR001,0,213,1,Gordon Henry St North Car Park,None,None,None,31,None
3,207210,741469862,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T05:51:02,6,Gordon Henry St North Car Park,207210TPR001,0,213,1,Gordon Henry St North Car Park,None,None,None,32,None
4,207210,741474196,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T07:03:16,6,Gordon Henry St North Car Park,207210TPR001,0,213,1,Gordon Henry St North Car Park,None,None,None,33,None


In [3]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2924545 entries, 0 to 2924544
Data columns (total 19 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   tsn                   int32 
 1   time                  int32 
 2   spots_x               int32 
 3   zones                 object
 4   ParkID                int32 
 5   occupancy             object
 6   MessageDate           object
 7   facility_id           int32 
 8   facility_name         object
 9   tfnsw_facility_id     object
 10  parent_zone_id        object
 11  spots_y               int32 
 12  zone_id               int32 
 13  zone_name             object
 14  occupancy.loop        object
 15  occupancy.monthlies   object
 16  occupancy.open_gate   object
 17  occupancy.total       int32 
 18  occupancy.transients  object
dtypes: int32(8), object(11)
memory usage: 334.7+ MB


Loading coordinates file:

In [4]:
coords = pd.read_json('./data/coords.json')
print(coords.info())

coords

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   facility_id  28 non-null     int64  
 1   longitude    28 non-null     float64
 2   latitude     28 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 804.0 bytes
None


,facility_id,longitude,latitude
0,6,-33.756009,151.154528
1,7,-34.672518,150.854695
2,8,-33.423883,151.341711
3,9,-33.952460,151.014838
4,10,-33.697777,151.300667
5,11,-33.713514,151.297315
6,12,-33.677276,151.305146
7,13,-33.752797,151.286485
8,14,-33.807172,151.090229
9,15,-34.031787,151.057190


Merging the coordinates with the main dataframe

In [7]:
# Merge the dataframes on 'facility_id'
merged_df = test.merge(coords, on='facility_id', how='left')
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2924545 entries, 0 to 2924544
Data columns (total 21 columns):
 #   Column                Dtype  
---  ------                -----  
 0   tsn                   int32  
 1   time                  int32  
 2   spots_x               int32  
 3   zones                 object 
 4   ParkID                int32  
 5   occupancy             object 
 6   MessageDate           object 
 7   facility_id           int32  
 8   facility_name         object 
 9   tfnsw_facility_id     object 
 10  parent_zone_id        object 
 11  spots_y               int32  
 12  zone_id               int32  
 13  zone_name             object 
 14  occupancy.loop        object 
 15  occupancy.monthlies   object 
 16  occupancy.open_gate   object 
 17  occupancy.total       int32  
 18  occupancy.transients  object 
 19  longitude             float64
 20  latitude              float64
dtypes: float64(2), int32(8), object(11)
memory usage: 379.3+ MB


In [9]:
merged_df['longitude'].isna().value_counts()

longitude
False    2924545
Name: count, dtype: int64

In [10]:
merged_df

,tsn,time,spots_x,zones,ParkID,occupancy,MessageDate,facility_id,facility_name,tfnsw_facility_id,...,spots_y,zone_id,zone_name,occupancy.loop,occupancy.monthlies,occupancy.open_gate,occupancy.total,occupancy.transients,longitude,latitude
0,207210,741448862,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T00:01:02,6,Gordon Henry St North Car Park,207210TPR001,...,213,1,Gordon Henry St North Car Park,None,None,None,31,None,-33.756009,151.154528
1,207210,741450092,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T00:21:32,6,Gordon Henry St North Car Park,207210TPR001,...,213,1,Gordon Henry St North Car Park,None,None,None,30,None,-33.756009,151.154528
2,207210,741468991,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T05:36:31,6,Gordon Henry St North Car Park,207210TPR001,...,213,1,Gordon Henry St North Car Park,None,None,None,31,None,-33.756009,151.154528
3,207210,741469862,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T05:51:02,6,Gordon Henry St North Car Park,207210TPR001,...,213,1,Gordon Henry St North Car Park,None,None,None,32,None,-33.756009,151.154528
4,207210,741474196,213,"[{'occupancy': {'loop': None, 'monthlies': Non...",1,"{'loop': None, 'monthlies': None, 'open_gate':...",2023-07-01T07:03:16,6,Gordon Henry St North Car Park,207210TPR001,...,213,1,Gordon Henry St North Car Park,None,None,None,33,None,-33.756009,151.154528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2924540,2126158,757341730,384,"[{'occupancy': {'loop': '106782', 'monthlies':...",1,"{'loop': '106782', 'monthlies': None, 'open_ga...",2023-12-31T23:42:10,33,Cherrybrook Car Park,2126158TPR001,...,384,1,SYD393 Cherrybrook Car Park,106782,None,None,272,None,-33.736703,151.031977
2924541,2126158,757341876,384,"[{'occupancy': {'loop': '106783', 'monthlies':...",1,"{'loop': '106783', 'monthlies': None, 'open_ga...",2023-12-31T23:44:36,33,Cherrybrook Car Park,2126158TPR001,...,384,1,SYD393 Cherrybrook Car Park,106783,None,None,271,None,-33.736703,151.031977
2924542,2126158,757341939,384,"[{'occupancy': {'loop': '106784', 'monthlies':...",1,"{'loop': '106784', 'monthlies': None, 'open_ga...",2023-12-31T23:45:39,33,Cherrybrook Car Park,2126158TPR001,...,384,1,SYD393 Cherrybrook Car Park,106784,None,None,270,None,-33.736703,151.031977
2924543,2126158,757342115,384,"[{'occupancy': {'loop': '106785', 'monthlies':...",1,"{'loop': '106785', 'monthlies': None, 'open_ga...",2023-12-31T23:48:35,33,Cherrybrook Car Park,2126158TPR001,...,384,1,SYD393 Cherrybrook Car Park,106785,None,None,269,None,-33.736703,151.031977


Saving `merged_df` to file:

In [11]:
merged_df.to_parquet('./data/carpark_history_6_months_zones_coords.parquet')

# Combining Dataframes

## 1. Loading carpark history data

In [ ]:
df_cp_history = pd.read_parquet('./data/carpark_history_6_months.parquet')
df_cp_history.head()

In [ ]:
df_cp_history.info()

## 2. Separating zones from carpark history
The zones will be included alongside the data for each day

In [ ]:
# Converting the zones column to its own dataframe
df_zones = pd.DataFrame(columns=['spots', 'zone_id', 'zone_name', 'parent_zone_id', 'occupancy.loop','occupancy.monthlies','occupancy.open_gate','occupancy.total','occupancy.transients'])
rename_format = {
    0: 'spots',
    1: 'zone_id',
    2: 'zone_name',
    3: 'parent_zone_id',
    4: 'occupancy_loops',
    5: 'occupancy_total',
    6: 'occupancy_monthlies',
    7: 'occupancy_open_gate',
    8: 'occupancy_transients'
}

zones_list = []

for index,row in df_cp_history.iterrows():
    # Normalize values in each record in zones column
    df_zone = pd.json_normalize(row['zones'])
    
    zones_list.append(df_zone)

# Concatendating zones list
df_zones = pd.concat(zones_list, ignore_index=True)

df_zones.head()

In [ ]:
df_zones.info()

Making a copy of zones before merging

In [ ]:
df_zones_copy = df_zones.copy()
df_cp_hist_copy = df_cp_history.copy()

In [ ]:
t_m = pd.merge(df_cp_hist_copy, df_zones_copy, how='outer',left_index=True, right_index=True)

t_m.head()

In [ ]:
t_m.info()

Changing columns to have appropriate data types

In [ ]:
cols_to_change = ['spots_y','zone_id','occupancy.total']

for col in cols_to_change:
    t_m[col] = t_m[col].astype(int)

t_m.info()

Loading the coordinates file:

In [ ]:
coords = pd.read_json('data/coords.json')
coords.head()

Saving merged file to its own document

In [ ]:
t_m.to_parquet('./data/carpark_history_6_months_with_zones.parquet')

Making a copy of the dataframe:

In [ ]:
tester = pd.read_parquet('./data/carpark_history_6_months_with_zones.parquet')

In [ ]:
tester.info()

Merging with tester dataframe

Mapping to main dataframe per facility ID

In [ ]:
# for i,r in coords.iterrows():
#   for index,row in tester.iterrows():
#     if row['facility_id'] == r['facility_id']:
#       t_m.at[index,'longitude'] = r['longitude']
#       t_m.at[index,'latitude'] = r['latitude']

# Merge the dataframes on 'facility_id'
merged_df = tester.merge(coords, on='facility_id', how='left')

# Update 'longitude' and 'latitude' columns where the condition is met
merged_df['longitude'].combine
tester['longitude'] = merged_df['longitude'].combine_first(tester['longitude'])
tester['latitude'] = merged_df['latitude'].combine_first(tester['latitude'])

# Other Info

## DATA UNDERSTANDING

## EXPLORATORY DATA ANALYSIS
### Univariate Data Analysis

The distribution of the numerical columns is visualized below using distribution plots (histogram with KDE) and boxplots.

In [ ]:
# Function to create varioous distribution plots for numeriacal columns
def create_distribution_plots(dataframe):
    # Check if the input is a DataFrame
    if not isinstance(dataframe, pd.DataFrame):
        raise ValueError("Input is not a DataFrame.")

    # Loop through each column in the DataFrame
    for column in dataframe.columns:
        # Check if the column is numerical
        if pd.api.types.is_numeric_dtype(dataframe[column]):
            # Create a figure with subplots
            fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 5))

            # Histogram with KDE
            sns.histplot(dataframe[column], kde=True, ax=axes[0])
            axes[0].set_title(f'Distribution Plot for {column}')
            axes[0].set_xlabel(column)
            axes[0].set_ylabel('Frequency')

            # Boxplot
            sns.boxplot(x=dataframe[column], ax=axes[1])
            axes[1].set_title(f'Boxplot for {column}')
            axes[1].set_xlabel(column)
            axes[1].set_ylabel('')

            # Violin plot
            sns.violinplot(x=dataframe[column], ax=axes[2])
            axes[2].set_title(f'Violin Plot for {column}')
            axes[2].set_xlabel(column)
            axes[2].set_ylabel('')

            plt.tight_layout()
            plt.show()


create_distribution_plots(no_duplicate_df)

In [ ]:
# Bar plot
plt.figure(figsize=(7, 4))
sns.barplot(x='day_of_week', y='parking_availability',
            data=no_duplicate_df, palette='viridis')
plt.title('Parking Availability by Day of the Week')
plt.xlabel('Day of the Week')
plt.ylabel('Parking Availability')
plt.show()

In [ ]:
# Convert 'time' to datetime format
no_duplicate_df['time'] = pd.to_datetime(no_duplicate_df['time'],format='%H:%M:%S',exact=True).dt.time

# Define time categories
def categorize_time(hour):
    if 4 <= hour < 7:
        return 'Early Morning'
    elif 7 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 15:
        return 'Afternoon'
    elif 15 <= hour < 18:
        return 'Late Afternoon'
    elif 18 <= hour < 20:
        return 'Evening'
    else:
        return 'Night'


# Create a new column 'time_category'
no_duplicate_df['time_category'] = no_duplicate_df['time'].apply(
    lambda x: categorize_time(x.hour))

# Bar plot grouped by time category
plt.figure(figsize=(7, 4))
sns.barplot(x='time_category', y='parking_availability',
            data=no_duplicate_df, palette='viridis')
plt.title('Parking Availability by Time Category')
plt.xlabel('Time Category')
plt.ylabel('Parking Availability')
plt.show()

In [ ]:
no_duplicate_df['facility_name']

In [ ]:
# Bar plot
plt.figure(figsize=(8, 8))
sns.barplot(y='facility_name', x='parking_availability',
            data=no_duplicate_df, palette='viridis',orient='h')
plt.title('Parking Availability by Parking Facility')
plt.xlabel('Facility Name')
# plt.xticks(rotation=45)
plt.ylabel('Parking Facility')
plt.show()

In [ ]:
# Create a pivot table for heatmap
heatmap_data = no_duplicate_df.pivot_table(
    values='parking_availability', index='day_of_week', columns='time_category', aggfunc='mean')

# Define the order of days of the week
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Define the order of time categories
time_order = ['Morning', 'Afternoon', 'Evening', 'Night']

# Create a heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data.loc[days_order, time_order], annot=True, cmap='viridis', fmt='.1f', linewidths=.5)
plt.title('Overall Parking Availability Trends')
plt.xlabel('Time Category')
plt.ylabel('Day of the Week')
plt.show()

In [ ]:
# Create a side-by-side boxplot or violin plot
plt.figure(figsize=(12, 6))

# Side-by-side boxplot
plt.subplot(1, 2, 1)
sns.boxplot(x='time_category', y='parking_availability',
            data=no_duplicate_df, palette='Set2')
plt.title('Parking Availability Distribution Across Time Intervals')

# Side-by-side violin plot
plt.subplot(1, 2, 2)
sns.violinplot(x='time_category', y='occupancy_total',
               data=no_duplicate_df, palette='Set2')
plt.title('Occupancy Rate Distribution Across Time Intervals')

plt.tight_layout()
plt.show()

In [ ]:
# Scatter plot
plt.figure(figsize=(8, 6))
sns.barplot(x='total_parking_spots', y='occupancy_total',
            data=no_duplicate_df, hue='facility_name', palette='Set2')
plt.title('Relationship Between Facility Capacity and Parking Demand')
plt.xlabel('Facility Capacity')
plt.ylabel('Parking Demand')
plt.legend(title='Facility')
plt.show()

In [ ]:
# Calculate average parking availability for each facility
avg_availability = no_duplicate_df.groupby(
    'facility_name')['parking_availability'].mean().reset_index()

# Plot a bar chart to visualize the distribution of average parking availability
plt.figure(figsize=(10, 6))
sns.barplot(y='facility_name', x='parking_availability', data=avg_availability, palette='Set2',orient='h')
plt.title('Average Parking Availability Across Facilities')
plt.xlabel('Facility')
plt.ylabel('Average Parking Availability')
plt.show()

# Identify facilities with consistently high or low parking availability
threshold = 70  # Adjust the threshold based on your criteria
high_availability_facilities = avg_availability[avg_availability['parking_availability'] >= threshold]['facility_name']
low_availability_facilities = avg_availability[avg_availability['parking_availability'] < threshold]['facility_name']

print(f'Facilities with consistently high parking availability: {list(high_availability_facilities)}')
print(f'Facilities with consistently low parking availability: {list(low_availability_facilities)}')

In [ ]:
# Calculate z-scores for parking availability
no_duplicate_df['z_score'] = (no_duplicate_df['parking_availability'] -
                              no_duplicate_df['parking_availability'].mean()) / no_duplicate_df['parking_availability'].std()

# Set a threshold for anomaly detection (e.g., z-score greater than 3)
anomaly_threshold = 3
anomalies = no_duplicate_df[no_duplicate_df['z_score'] > anomaly_threshold]

# Visualize anomalies in a scatter plot
plt.figure(figsize=(10, 6))
sns.scatterplot(x='date', y='parking_availability',
                data=no_duplicate_df, label='Normal', color='blue')
sns.scatterplot(x='date', y='parking_availability', data=anomalies, label='Anomalies', color='red', s=100)
plt.title('Anomalous Patterns in Parking Availability')
plt.xlabel('Date')
plt.xticks(rotation=90)
plt.ylabel('Parking Availability')
plt.legend()
plt.show()

In [ ]:
no_duplicate_df

In [ ]:
carparks_df_copy.loc[carparks_df_copy.parking_availability == 0]